Consept: you just upload table with schedule in the program and then pandas will analyse it for useful info

upload table from your computer:

In [ ]:
from google.colab import files
uploaded_data = files.upload()

Saving ИИТ_2к_20-21_весна.xlsx to ИИТ_2к_20-21_весна.xlsx


In [ ]:
import pandas as pd

table = pd.read_excel('/content/ИИТ_2к_20-21_весна.xlsx')
table

,Unnamed: 0,Р А С П И С А Н И Е занятий весеннего семестра \n2 курса Института ИТ на 2020-2021 уч. г.,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,УТВЕРЖДАЮ \nСоветник по УМР В.Л. Панков\n 25.01.2021 .,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Р А С П И С А Н И Е занятий весеннего семестра \n2 курса Института ИТ на 2020-2021 уч. г. .1,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,УТВЕРЖДАЮ \nСоветник по УМР В.Л. Панков\n 25.01.2021 ..1,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 320,Р А С П И С А Н И Е занятий весеннего семестра \n2 курса Института ИТ на 2020-2021 уч. г. .16,Unnamed: 322,Unnamed: 323,Unnamed: 324,Unnamed: 325,Unnamed: 326,Unnamed: 327,Unnamed: 328,Unnamed: 329,Unnamed: 330,Unnamed: 331,Unnamed: 332,Unnamed: 333,Unnamed: 334,УТВЕРЖДАЮ \nСоветник по УМР В.Л. Панков\n 25.01.2021 ..16,Unnamed: 336,Unnamed: 337,Unnamed: 338,Unnamed: 339,Unnamed: 340,Р А С П И С А Н И Е занятий весеннего семестра \n2 курса Института ИТ на 2020-2021 уч. г. .17,Unnamed: 342,Unnamed: 343,Unnamed: 344,Unnamed: 345,Unnamed: 346,Unnamed: 347,Unnamed: 348,Unnamed: 349,Unnamed: 350,Unnamed: 351,Unnamed: 352,Unnamed: 353,Unnamed: 354,УТВЕРЖДАЮ \nСоветник по УМР В.Л. Панков\n 25.01.2021 ..17,Unnamed: 356,Unnamed: 357,Unnamed: 358,Unnamed: 359
0,День недели,Группа,NaN,NaN,NaN,ИНБО-01-19,ППИ,09.03.03\nИО,29,NaN,ИНБО-04-19,ППИ,09.03.03\nИО,27,NaN,ИНБО-07-19,ППИ,09.03.03\nИО,28,NaN,День недели,Группа,NaN,NaN,NaN,ИНБО-10-19,ППИ,09.03.03\nИО,26,NaN,ИНБО-02-19,ППИ\n239,09.03.03\nИО,32,NaN,ИНБО-03-19,ППИ\n256,09.03.03\nИО,25,NaN,...,День недели,Группа,NaN,NaN,NaN,ИКБО-07-19,МОСИТ,09.03.04\nСПИ,29,NaN,ИКБО-11-19,МОСИТ,09.03.04\nСПИ,28,NaN,ИКБО-12-19,МОСИТ,09.03.04\nСПИ,29,NaN,День недели,Группа,NaN,NaN,NaN,ИКБО-25-19,МОСИТ,09.03.04\nСПИ,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,№ пары,Нач.\nзанятий,Оконч.\nзанятий,Неделя,Предмет,Вид\nзанятий,ФИО преподавателя,№ \nауд.,Ссылка,Предмет,Вид\nзанятий,ФИО преподавателя,№ \nауд.,Ссылка,Предмет,Вид\nзанятий,ФИО преподавателя,№ \nауд.,Ссылка,NaN,№ пары,Нач.\nзанятий,Оконч.\nзанятий,Неделя,Предмет,Вид\nзанятий,ФИО преподавателя,№ \nауд.,Ссылка,Предмет,Вид\nзанятий,ФИО преподавателя,№ \nауд.,Ссылка,Предмет,Вид\nзанятий,ФИО преподавателя,№ \nауд.,Ссылка,...,NaN,№ пары,Нач.\nзанятий,Оконч.\nзанятий,Неделя,Предмет,Вид\nзанятий,ФИО преподавателя,№ \nауд.,Ссылка,Предмет,Вид\nзанятий,ФИО преподавателя,№ \nауд.,Ссылка,Предмет,Вид\nзанятий,ФИО преподавателя,№ \nауд.,Ссылка,NaN,№ пары,Нач.\nзанятий,Оконч.\nзанятий,Неделя,Предмет,Вид\nзанятий,ФИО преподавателя,№ \nауд.,Ссылка,Предмет,Вид\nзанятий,ФИО преподавателя,№ \nауд.,Ссылка,Предмет,Вид\nзанятий,ФИО преподавателя,№ \nауд.,Ссылка
2,ПОНЕДЕЛЬНИК,1,9-00,10-30,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ПОНЕДЕЛЬНИК,1,9-00,10-30,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ПОНЕДЕЛЬНИК,1,9-00,10-30,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ПОНЕДЕЛЬНИК,1,9-00,10-30,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2,10-40,12-10,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,10-40,12-10,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2,10-40,12-10,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,10-40,12-10,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

Us you see this table is unreadeble. We must do something to fix it.

In [ ]:
!pip install python-telegram-bot

In [ ]:
!pip install telegramcalendar

ERROR: Could not find a version that satisfies the requirement telegramcalendar (from versions: none)
ERROR: No matching distribution found for telegramcalendar


In [ ]:
!wget https://github.com/unmonoqueteclea/calendar-telegram/blob/master/telegramcalendar.py

--2021-02-20 15:17:12--  https://github.com/unmonoqueteclea/calendar-telegram/blob/master/telegramcalendar.py
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘telegramcalendar.py’

telegramcalendar.py     [ <=>                ] 137.24K  --.-KB/s    in 0.04s   

2021-02-20 15:17:12 (3.50 MB/s) - ‘telegramcalendar.py’ saved [140531]



In [ ]:
import logging
from telegram.ext import Updater
from telegram.ext import  CallbackQueryHandler
from telegram.ext import  CommandHandler
from telegram import  ReplyKeyboardRemove


import telegramcalendar


TOKEN = "1129048113:AAGpp0FPckxMzROx6pFldr5CNORkW0CXnfM"


logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

def calendar_handler(bot,update):
    update.message.reply_text("Please select a date: ",
                        reply_markup=telegramcalendar.create_calendar())


def inline_handler(bot,update):
    selected,date = telegramcalendar.process_calendar_selection(bot, update)
    if selected:
        bot.send_message(chat_id=update.callback_query.from_user.id,
                        text="You selected %s" % (date.strftime("%d/%m/%Y")),
                        reply_markup=ReplyKeyboardRemove())



up = Updater(TOKEN)

up.dispatcher.add_handler(CommandHandler("calendar",calendar_handler))
up.dispatcher.add_handler(CallbackQueryHandler(inline_handler))

up.start_polling()
up.idle()

2021-02-20 15:17:57,946 - apscheduler.scheduler - INFO - Scheduler started
2021-02-20 15:18:21,469 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-02-20 15:18:21,471 - apscheduler.scheduler - INFO - Scheduler has been shut down


## Сам бот

1. Научим его понимать разные сообщения 
2. Добавим необходимые базовые функции (в режиме переписки)
3. Научим новым командам и сделаем календарь
4. Внедряем расписание

In [71]:
!pip install telebot
!pip install PyTelegramBotAPI

In [84]:
import random
import telebot
import nltk
from datetime import date

bot = telebot.TeleBot(TOKEN)  

TOKEN = '1129048113:AAGpp0FPckxMzROx6pFldr5CNORkW0CXnfM'
BOT_CONFIG = {
    'intents': {
        
        'hello': {
            'examples': ['Приветик', 'Добрый день', 'Шалом', 'Привет, бот', 'Привет'],
            'responses': ['Привет, меня зовут Варфоломей', 
                          'Привет! Рад тебя видеть',
                          'Добрый день, ты как всегда выглядишь круто']
        },
        
        'bye': {
            'examples': ['Пока', 'Спокойной ночи', 'Досвидания', 'До встречи'],
            'responses': ['Удачи! Если что, пиши. Я всегда здесь','Буду тебя ждать ^^']
        },
        
        'name': {
            'examples': ['Как тебя зовут?', 'Скажи свое имя?', 'Твое имя?'],
            'responses':['Меня зовут Варфоломей','Я красная панда Варфоломей']
        },
    },

    'failure_phrases': [
                      'Ничего не понимаю. Перефразируй, пожалуйста',
                     'Скажи фразу другими словами, я тебя не понял :(',
                     'Я только учусь, спроси что-нибудь другое',
                     'Я в неведении, спроси у гугла. Передавай от меня привет',
                     'Может тебе стоит спросить у Алисы?',
                     'Не знаю, что сказать',
                     'А ты знаешь, что в Нью-Джерси все коты должны носить колокольчики, чтобы птицы всегда знали об их расположении? Так какой вопрос ты задал?'
    ]
}

def get_failure_phrase():
  failure_phrases = BOT_CONFIG['failure_phrases']
  return random.choice(failure_phrases)

def generate_answer(replica):
  return None

def get_answer_by_intent(intent):
  if intent in BOT_CONFIG['intents']:
    responses = BOT_CONFIG['intents'][intent]['responses']
    return random.choice(responses)

def classify_intent(replica):
  replica = clear_phrase(replica)

  for intent, intent_data in BOT_CONFIG['intents'].items():
    for example in intent_data['examples']:
      example = clear_phrase(example)
      distance = nltk.edit_distance(replica, example)
      if distance / len(example) < 0.4:
        return intent

def clear_phrase(phrase):
  phrase = phrase.lower()
  alphabet = 'ёйцукенгшщзхъэждлорпавыфячсмитьбю- '
  result = ''.join(symbol for symbol in phrase if symbol in alphabet)
  return result

def bot_ans(replica):
  # NLU - intent classificator
  intent = classify_intent(replica)

  # Answer generation
  # Выбор заготовленной реплики
  if intent:
    answer = get_answer_by_intent(intent)
    if answer:
      return answer

  #вызов генеративной модели
  answer = generate_answer(replica)
  if answer:
    return answer

  # заглушки
  return get_failure_phrase()

@bot.message_handler(commands=['start'])
def Send_Welcome(message):
  bot.reply_to(message, "Привет, {0}! меня зовут Варфоламей. Надеюсь, мы подружимся ^^".format(message.from_user.first_name))

@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    bot.send_message(message.from_user.id, bot_ans(message.text))

bot.polling(none_stop=True)